# Exploring data

In [1]:
import pandas as pd
import numpy as np 
import warnings; warnings.simplefilter('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from ast import literal_eval
from matplotlib import pyplot as plt
from nltk.stem.snowball import SnowballStemmer

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

## Simple Recommender
### Sort movie by its popularity
How to calculate movie popularity? 

According to imdb, movie rating is calculated as follow:

Weight Rated = (v/(m+v)*R) + (m/(v+m)*C)

For v = numbers of votes for the movie

For m = The minimum votes required to be on the chart

For R = The average rating of the movie 

For C = The mean vote across the whole record

### Request data from database




In [2]:
import requests
import pandas as pd

url = "https://api.themoviedb.org/3/movie/{}"
latest_movieID = 528128
first_movieID = 1
D = {'api_key':'7cfc8557ee5d234bff85e81eb103c210'}
api = requests.get(url.format(latest_movieID) , params = D)
movie_json = api.json()
list(movie_json.keys())


['adult',
 'backdrop_path',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

In [3]:
# import time
# #query every movive for the movie database 

# for index in range (4138, 13203):
#     movie_list= []
#     for movieID in range(0,40):
#         api = requests.get(url.format(movieID + index*40),params = D)
#         print(movieID + index*100)
#         if(api.status_code == 404):
#             print("ERROR 404: MOVIE NOT FOUND")
#         elif(api.status_code == 200):
#             print("SUCCESS")
#             movie = api.json()
#             movie_list.append(movie)
#         else:
#             print(api.status_code)
#     if(len(movie_list) != 0):
#         movie_data = pd.DataFrame(movie_list)
#         average_rate = movie_data.vote_average.mean()
#         movie_data['average_rate'] = average_rate
#         csv_file = "movie_data{}.csv"
#         movie_data.to_csv(csv_file.format(index))
#     time.sleep(10)

In [4]:
# import warnings; warnings.simplefilter('ignore')

movie_data = pd.read_csv("data/md.csv")
link_id = pd.read_csv("data/links_small.csv")
link_id = link_id[['movieId','imdbId', 'tmdbId']]

In [5]:
#m : minimum vote to be in the list 
m = movie_data['vote_count'].quantile(0.95)

#C: mean vote accross every record
C = movie_data['vote_average'].mean()

def weightRate(movie):
    #v : number of vote 
    v = movie['vote_count']
    #R : average rating of a movie 
    R = movie['vote_average']
    return R*v/(v+m) + C*m/(v+m)
    
movie_data['weight_rate'] = movie_data.apply(weightRate, axis = 1)
mdRanked = movie_data.sort_values(by= ['weight_rate'], ascending = False)

## Content based recommender
### What's the different 
This recommender system based on the used preference, more details of its, look at the google docs sheet 


In [6]:
link_id = link_id.astype(int)

tmdbId = link_id[link_id['tmdbId'].notnull()]['tmdbId'].astype(int)
movie_data = movie_data[movie_data['id'].notnull()]
movie_data['id']= movie_data['id'].astype(int)

In [7]:
# movie_data.to_csv('movie_data.csv')
tmdb_md = movie_data[movie_data['id'].isin(tmdbId)]
tmdb_md['tagline'] = tmdb_md['tagline'].fillna('')
tmdb_md['overview'] = tmdb_md['overview'].fillna('')
tmdb_md['description'] = tmdb_md['tagline'] + tmdb_md['overview']

tmdb_md.shape

(7082, 12)

In [9]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [10]:
tmdb_md = tmdb_md.reset_index()

In [11]:
titles = tmdb_md['title']
    
indices = pd.Series(data = tmdb_md.index, index =titles )

def getRecommendation(title):
    title_index = indices[title]
    sim_score = list(enumerate(cosine_sim[title_index]))
    sim_score = sorted(sim_score, key = lambda x: x[1], reverse = True)
    sim_score = sim_score[1:31]
    movie_indice = [i[0] for i in sim_score]
    
    return indices.iloc[movie_indice] 

In [12]:
getRecommendation('The Dark Knight')    

title
The Dark Knight Rises                        89
Batman Forever                             6975
Batman Returns                              986
Batman: Under the Red Hood                  611
Batman: The Dark Knight Returns, Part 2     509
Batman                                      488
Batman: Year One                           1474
Batman: Mask of the Phantasm               1368
JFK                                         609
Batman: The Dark Knight Returns, Part 1     620
Sherlock Holmes: A Game of Shadows          403
Batman Begins                               126
Q & A                                      4620
Law Abiding Citizen                         412
The File on Thelma Jordon                  5336
Flying Tigers                              5451
Night Falls on Manhattan                   4563
The Young Savages                          5702
Batman v Superman: Dawn of Justice         4147
Batman & Robin                             7080
Rush Hour 3                       

In [14]:
credits = pd.read_csv("data/credits.csv")
keywords = pd.read_csv("data/keywords.csv")

credits['id'] = credits['id'].astype(int)
keywords['id'] = keywords['id'].astype(int)
movie_data['id'] = movie_data['id'].astype(int)

In [15]:
movie_data =pd.merge(movie_data, credits, how = "inner", on= 'id')
movie_data = pd.merge(movie_data, keywords,how = "inner", on = 'id')

In [16]:
movie_data['cast'] = movie_data['cast'].apply(literal_eval)
movie_data['crew'] = movie_data['crew'].apply(literal_eval)
movie_data['keywords'] = movie_data['keywords'].apply(literal_eval)

In [18]:
def getActor(x):
    if (len(x) >5):
        x = x[:5]
    else:
        x = x
    return [actor['name'].replace(" ","") for actor in x ]

def getDirector(x):
    return [crew['name'].replace(" ", "") for crew in x if crew['job'] == 'Director']

def getKeywords(x):
    return [key['name'] for key in x]


tmdb_movie = movie_data[movie_data['id'].isin(tmdbId)]
tmdb_movie['actors'] =tmdb_movie['cast'].apply(getActor)
tmdb_movie['director'] = tmdb_movie['crew'].apply(getDirector)
tmdb_movie['keywords'] = tmdb_movie['keywords'].apply(getKeywords)

In [19]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')
tmdb_movie['keywords'] = tmdb_movie['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])

In [21]:
tmdb_movie['details'] = tmdb_movie['keywords'] + tmdb_movie['director'] + tmdb_movie['actors']
tmdb_movie['details'] = tmdb_movie['details'].apply (lambda x: ' '.join(x))
# tmdb_movie.to_csv('tmdb_movie.csv')

itali love at first sight loss of fath patriarch organized crim mafia lawyer italian american crime famili rise to pow mob boss 1940s FrancisFordCoppola MarlonBrando AlPacino JamesCaan RichardS.Castellano RobertDuvall


In [22]:
tf_details = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tf_matrix = tf.fit_transform(tmdb_movie['details'])
cosine_sim = linear_kernel(tf_matrix, tf_matrix)
cosine_sim.shape

(7252, 7252)

In [23]:
tmdb_movie = tmdb_movie.reset_index()

In [25]:
titles = tmdb_movie['title']    
indices = pd.Series(data = tmdb_movie.index, index =titles )

In [26]:
getRecommendation('The Dark Knight')

title
Batman Begins                               128
The Dark Knight Rises                        91
Batman: Under the Red Hood                  615
Batman Returns                              994
Batman                                      492
Batman & Robin                             7250
Superman                                    773
Batman v Superman: Dawn of Justice         4233
Superman III                               6981
Justice League: Crisis on Two Earths       2092
Man of Steel                                912
Wonder Woman                               2150
Batman: Mask of the Phantasm               1400
Batman: Year One                           1506
Batman Forever                             7145
Kick-Ass 2                                 1449
Batman: The Dark Knight Returns, Part 2     513
All Star Superman                          2679
Superman II                                1580
Batman: The Dark Knight Returns, Part 1     624
Justice League: The Flashpoint Par